# Guide to instantiating models with EUGENe
**Authorship:**
Adam Klie (last updated: *06/22/2023*)
***
**Description:**
This notebook is meant to serve as a guide to instantiating models with EUGENe. It will cover the following topics:
1. EUGENe's `models` library
2. Using EUGENe's built-in models
3. Using configuration files to instantiate models
4. Using custom architectures
5. Using custom LightningModules
***

In [5]:
# General imports
import torch

In [13]:
# Give me some dummy data
x_linear = torch.randn(10, 4)
x = torch.randn(10, 4, 100)

## EUGENe's `models` library -- the basics

### Stating with PyTorch

I like to think of architecting neural networks as playing with adult legos. Legos specifically designed for geeks who like to learn from data. 

Designing and training neural networks for regulatory genomics requires a comprehensive library of architecture lego blocks. Fortunately for us, PyTorch provides an extensive library that we can use right out of the box. 

In [14]:
import torch.nn as nn

In [15]:
# We define a simple linear layer with 4 inputs and 5 outputs.
layer = nn.Linear(4, 5)
layer_out = layer(x_linear)
layer, layer_out.shape

(Linear(in_features=4, out_features=5, bias=True), torch.Size([10, 5]))

### EUGENe's layers build on PyTorch

EUGENe builds on PyTorch by adding several useful layers such as inception and residual layers. These are define in the `eugene.models.base._layers` module. 

In [9]:
# Load EUGENe's layers 
from eugene.models.base import _layers as layers

Layers in EUGENe can be broken up into several categories: **Activations, Convolutional, Pooling, Recurrent, Attention, Normalizer, Wrappers, Gluers, Sampling, Noise, Misc**

In [16]:
# Take exponential activations as an example that have been shown to improve the interpretability of models
layer = layers.Exponential(inplace=False)
layer_out = layer(x)
layer, layer_out.shape

(Exponential(), torch.Size([10, 4, 100]))

In [21]:
# For a more complex example, we can use the InceptionConv1D layer that uses multiple sizes of convolutions in a single layer
layer = layers.InceptionConv1D(in_channels=4, out_channels=16)
layer_out = layer(x)
layer, layer_out.shape

(InceptionConv1D(
   (conv1): Conv1d(4, 4, kernel_size=(1,), stride=(1,))
   (conv2_1): Conv1d(4, 4, kernel_size=(1,), stride=(1,))
   (conv2_2): Conv1d(4, 4, kernel_size=(3,), stride=(1,), padding=same)
   (conv3_1): Conv1d(4, 4, kernel_size=(1,), stride=(1,))
   (conv3_2): Conv1d(4, 4, kernel_size=(5,), stride=(1,), padding=same)
   (maxpool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
   (conv_maxpool): Conv1d(4, 4, kernel_size=(1,), stride=(1,))
 ),
 torch.Size([10, 16, 100]))

In [22]:
# We can also use the MultiHeadAttention layer that is used in the Transformer architecture
layer = layers.MultiHeadAttention(
    input_dim=4,
    head_dim=10,
    num_heads=2
)
layer_out = layer(x.transpose(1, 2), mask=None)
layer, layer_out.shape

(MultiHeadAttention(
   (qkv): Linear(in_features=4, out_features=60, bias=False)
   (softmax): Softmax(dim=-1)
   (dropout_layer): Dropout(p=0.0, inplace=False)
   (projection_layer): Sequential(
     (0): Linear(in_features=20, out_features=4, bias=True)
     (1): Dropout(p=0.0, inplace=False)
   )
 ),
 torch.Size([10, 100, 4]))

Remember you can always bring up the signature and docstring for any layer by using the `?` operator in Jupyter notebooks. For example, to bring up the docstring for the `Inception` layer, you can run the following code:

```python
from eugene.models.base._layers import InceptionConv1d
InceptionConv1d?
```

In [29]:
# One more common example of what we call a wrapper layer. Residual layers are used to add the input to the output of a layer.
conv_layer = torch.nn.Conv1d(4, 4, 5, padding="same")
layer = layers.Residual(conv_layer)
layer_out = layer(x)
layer, layer_out.shape

(Residual(
   (wrapped): Conv1d(4, 4, kernel_size=(5,), stride=(1,), padding=same)
 ),
 torch.Size([10, 4, 100]))

### From layers to blocks

Additionally, EUGENe introduces flexible functions for establishing common “blocks” that are composed of heterogeneous sets of layers arranged in a predefined or adaptable order. Blocks are available in the `eugene.models.base._blocks` module.

In [30]:
from eugene.models.base import _blocks as blocks

A convolutional block (Conv1DBlock in EUGENe) often comprises convolutional, normalization, activation, and dropout layers in different orderings depending on the model and task.

In [ ]:
conv1d_block = blocks.Conv1DBlock(
    input_len=100,
    input_channels=4,
    output_channels=32,
    conv_kernel=23,
    dropout_rate=0.1,
)
block_out = conv1d_block(x)
conv1d_block, block_out.shape

Note! Order matters! From model to model, the order of the layers can be employed differently. For example, a DeepSEA conv block uses the ordering “conv-act-pool-dropout”, while a DeepSTARR conv block uses “conv-norm-act-pool”. We can change the ordering of the layers in the conv block using the order argument.

In [40]:
# A la DeepSEA, note that we can also omit one of the layers by simply not including it in the order argument. We've also added other arguments that are flexible in EUGENe
conv1d_block = blocks.Conv1DBlock(
    input_len=100,
    input_channels=4,
    output_channels=32,
    conv_kernel=23,
    conv_type="conv1d",
    conv_padding="same",
    pool_type="max",
    norm_type="batchnorm",
    dropout_rate=0.5,
    order="conv-act-pool-dropout"
)
block_out = conv1d_block(x)
conv1d_block, block_out.shape

(Conv1DBlock(
   (layers): Sequential(
     (conv): Conv1d(4, 32, kernel_size=(23,), stride=(1,), padding=same)
     (act): ReLU()
     (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
     (dropout): Dropout(p=0.5, inplace=False)
   )
 ),
 torch.Size([10, 32, 100]))

In [41]:
# A la DeepSTARR
conv1d_block = blocks.Conv1DBlock(
    input_len=100,
    input_channels=4,
    output_channels=32,
    conv_kernel=23,
    conv_type="conv1d",
    conv_padding="same",
    pool_type="max",
    norm_type="batchnorm",
    dropout_rate=0.5,
    order="conv-norm-act-pool"
)
block_out = conv1d_block(x)
conv1d_block, block_out.shape

(Conv1DBlock(
   (layers): Sequential(
     (conv): Conv1d(4, 32, kernel_size=(23,), stride=(1,), padding=same)
     (norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (act): ReLU()
     (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
   )
 ),
 torch.Size([10, 32, 100]))

We also offer 2 other blocks: `DenseBlocks`, `RecurrentBlocks`. We leave it as an exercise to the reader to explore these blocks or to create their own! (Transformer blocks coming soon!)

### From blocks to towers

We can also stack multiple blocks on top of one another so that we can build very deep architectures without having to write a lot of code. We call these stacks of blocks "towers". Towers are available in the `eugene.models.base._towers` module.

In [43]:
from eugene.models.base import _towers as towers

We built a single flexible `Tower` class that can handle any block. The class takes in a set of `static_block_args` that you want to repeat across blocks and a set of `dynamic_block_args` that you want to change for each block. You can also pass in a set of `mults` that you want to use to scale an argument with in each block (e.g. if you want to do exponentially increasing dilations like in Basenji). We show an example of using the `Tower` class with a Conv1DBlock (probably the most common in genomics) below:

In [48]:
from eugene.models.base import _blocks as blocks

In [50]:
# Here we toss a Conv1DBlock with some different arguments than before. If you like dogs, then this tower is for you.
tower = towers.Tower(
    input_size=(4, 100),
    block=blocks.Conv1DBlock,
    repeats=3,
    static_block_args={'input_len': 100, 'conv_kernel': 3, 'conv_padding': 'same', 'conv_type': 'conv1d', 'activation': 'gelu', 'order': 'conv-norm-act-dropout-pool'},
    dynamic_block_args={'input_channels': [4, 10, 20], 'output_channels': [10, 20, 30]},
    mults={"conv_dilation": 2}
)
tower_out = tower(x)
tower, tower.input_size, tower.output_size, tower_out.shape

(Tower(
   (blocks): Sequential(
     (conv1dblock_0): Conv1DBlock(
       (layers): Sequential(
         (conv): Conv1d(4, 10, kernel_size=(3,), stride=(1,), padding=same)
         (norm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act): GELU()
         (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
       )
     )
     (conv1dblock_1): Conv1DBlock(
       (layers): Sequential(
         (conv): Conv1d(10, 20, kernel_size=(3,), stride=(1,), padding=same, dilation=(2,))
         (norm): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (act): GELU()
         (pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
       )
     )
     (conv1dblock_2): Conv1DBlock(
       (layers): Sequential(
         (conv): Conv1d(20, 30, kernel_size=(3,), stride=(1,), padding=same, dilation=(4,))
         (norm): BatchNorm1d(30, eps=1e-05, momentum=0.1, a

The other cool thing about this is it is not EUGENe specific! That is, it isn't constrained to working on EUGENe layers or blocks. You can use it with any PyTorch layer or block!

In [45]:
# Here we just repeat a linear layer 3 times, decreasing the number of units in each layer
tower = towers.Tower(
    input_size=400,
    block=torch.nn.Linear,
    repeats=3,
    dynamic_block_args={'in_features': [400, 200, 100], 'out_features': [200, 100, 10]},
)
tower_out = tower(x.reshape(10, -1))
tower, tower.input_size, tower.output_size, tower_out.shape

(Tower(
   (blocks): Sequential(
     (linear_0): Linear(in_features=400, out_features=200, bias=True)
     (linear_1): Linear(in_features=200, out_features=100, bias=True)
     (linear_2): Linear(in_features=100, out_features=10, bias=True)
   )
 ),
 400,
 [10],
 torch.Size([10, 10]))

Using standarized objects such as blocks and towers has several benefits:
1. It makes it easier to write code, and for others to read your code.
2. It makes it easier to interpret models. You can pull layers out of the model based on the standard names and see what is going on.
3. It makes it easier to debug models and identify where things are going wrong.

## Using EUGENe's built-in models

### Architectures as simple nn.Modules

We refer to architectures in EUGENe as any class that inherits from `torch.nn.Module` and includes a `forward` method. This is the standard way of defining architectures in PyTorch. This means that any of the layers, blocks, or towers we discussed are technically architectures on their own. However, architectures that are actually used in practice are usually composed of multiple layers, blocks, and towers in combination.

In [51]:
import torch.nn as nn

In [63]:
class EverythingEverywhereAllAtOnce(torch.nn.Module):
    def __init__(
        self
    ):
        super(EverythingEverywhereAllAtOnce, self).__init__()
        self.layer = nn.Linear(4, 5)
        self.layer2 = layers.Exponential(inplace=False)
        self.layer3 = layers.InceptionConv1D(in_channels=4, out_channels=16)
        self.layer4 = layers.MultiHeadAttention(
            input_dim=4,
            head_dim=10,
            num_heads=2
        )
        self.layer5 = layers.Residual(conv_layer)
        self.layer6 = blocks.Conv1DBlock(
            input_len=100,
            input_channels=4,
            output_channels=32,
            conv_kernel=23,
            dropout_rate=0.1,
        )
        self.layer7 = towers.Tower(
            input_size=(4, 100),
            block=blocks.Conv1DBlock,
            repeats=3,
            static_block_args={'input_len': 100, 'conv_kernel': 3, 'conv_padding': 'same', 'conv_type': 'conv1d', 'activation': 'gelu', 'order': 'conv-norm-act-dropout-pool'},
            dynamic_block_args={'input_channels': [4, 10, 20], 'output_channels': [10, 20, 30]},
            mults={"conv_dilation": 2}
        )
        self.layer8 = towers.Tower(
            input_size=400,
            block=torch.nn.Linear,
            repeats=3,
            dynamic_block_args={'in_features': [400, 200, 100], 'out_features': [200, 100, 10]},
        )

    def forward(self, x):
        x = self.layer(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        return x

In [69]:
# Note, the forward for this architecture will not actually work, I just wanted to stack everything we've looked at so far into one model
EverythingEverywhereAllAtOnce()

EverythingEverywhereAllAtOnce(
  (layer): Linear(in_features=4, out_features=5, bias=True)
  (layer2): Exponential()
  (layer3): InceptionConv1D(
    (conv1): Conv1d(4, 4, kernel_size=(1,), stride=(1,))
    (conv2_1): Conv1d(4, 4, kernel_size=(1,), stride=(1,))
    (conv2_2): Conv1d(4, 4, kernel_size=(3,), stride=(1,), padding=same)
    (conv3_1): Conv1d(4, 4, kernel_size=(1,), stride=(1,))
    (conv3_2): Conv1d(4, 4, kernel_size=(5,), stride=(1,), padding=same)
    (maxpool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (conv_maxpool): Conv1d(4, 4, kernel_size=(1,), stride=(1,))
  )
  (layer4): MultiHeadAttention(
    (qkv): Linear(in_features=4, out_features=60, bias=False)
    (softmax): Softmax(dim=-1)
    (dropout_layer): Dropout(p=0.0, inplace=False)
    (projection_layer): Sequential(
      (0): Linear(in_features=20, out_features=4, bias=True)
      (1): Dropout(p=0.0, inplace=False)
    )
  )
  (layer5): Residual(
    (wrapped): Conv1d(4, 4, k

### Built-in architectures

We could just leave you on your own to play with your new lego sets, but for those who want some fully assembled models to play with, we have you covered. We've built a model zoo (cliche I know) of models that are available in the `eugene.models.zoo` module.

In [70]:
from eugene.models import zoo

Like we mentioned before, every architecture in the zoo is a class that inherits from `torch.nn.Module` and includes a `forward` method. This means that you can use any of the architectures in the zoo just like you would any other PyTorch model.

### Types of built-in architectures

We are ever the [splitters at EUGENe](https://en.wikipedia.org/wiki/Lumpers_and_splitters), so we split our models into many categories based on the types of tasks they were designed for. The model zoo has the following sections:

1. Basic models
2. Transcription factor binding prediction models
3. Regulatory classification models
4. Cis-regulatory element (CRE) activity prediction models
5. Profile predictors
6. Single cell predictors

#### Basic architectures

Let's start by checking out the basic models

In [73]:
from eugene.models.zoo._basic_models import FCN, CNN, RNN, Hybrid

We provide customizable fully connected (FCN), convolutional (CNN), recurrent (RNN) and hybrid (a combination of the three) architectures that can all be instantiated from single function calls! Did I say we were splitters?

In [75]:
# FCNs are just a bunch of linear layers stacked on top of each other
model = FCN(
    input_len=100,
    output_dim=10,
    dense_kwargs={
        "hidden_dims": [50, 25],
        "activations": "relu",
        "batchnorm": True,
    }
)
model_out = model(x)
model, model_out.shape

(FCN(
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=400, out_features=50, bias=True)
       (1): ReLU()
       (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (3): Linear(in_features=50, out_features=25, bias=True)
       (4): ReLU()
       (5): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (6): Linear(in_features=25, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

In [76]:
# Here is just another example of an FCN, this time with a different activation function and dropout, as well as other customizable parameters
model = FCN(
    input_len=100,
    output_dim=1,
    dense_kwargs=dict(
        hidden_dims=[100, 50, 25], 
        activations=["relu", None, None], 
        dropout_rates=[0.1, 0.5], 
        batchnorm=True, 
        batchnorm_first=True, 
        biases=False
    ),
)
model_out = model(x)
model, model_out.shape

(FCN(
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=400, out_features=100, bias=False)
       (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (2): ReLU()
       (3): Dropout(p=0.1, inplace=False)
       (4): Linear(in_features=100, out_features=50, bias=False)
       (5): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (6): Dropout(p=0.5, inplace=False)
       (7): Linear(in_features=50, out_features=25, bias=False)
       (8): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (9): Linear(in_features=25, out_features=1, bias=False)
     )
   )
 ),
 torch.Size([10, 1]))

In [78]:
# Now we can move to CNNs, which are just a bunch of convolutions stacked on top of each other, followed by a dense block
model = CNN(
    input_len=100,
    output_dim=10,
    conv_kwargs={
        "input_channels": 4,
        "conv_channels": [10, 10],
        "conv_kernels": [5, 3],
        "activations": [],
        "pool_types": []
    }
)
model_out = model(x)
model, model_out.shape

(CNN(
   (conv1d_tower): Conv1DTower(
     (layers): Sequential(
       (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
       (1): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
     )
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=940, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

Note that the number of incoming features to the dense block is determined automagically! Thanks [torchinfo](https://github.com/TylerYep/torchinfo)!

In [79]:
# Let's skip RNNs since those aren't used a whole lot and jump to Hybrids, which are just convolutions followed by recurrent layers and then a dense block
model = Hybrid(
    input_len=100,
    output_dim=10,
    conv_kwargs={
        "input_channels": 4,
        "conv_channels": [10, 10],
        "conv_kernels": [5, 3],
        "activations": "relu",
        "pool_types": "max"
    },
    recurrent_kwargs={
        "hidden_dim": 10,
        "num_layers": 10,
        "bidirectional": True
    }
)
model_out = model(x)
model, model_out.shape


(Hybrid(
   (conv1d_tower): Conv1DTower(
     (layers): Sequential(
       (0): Conv1d(4, 10, kernel_size=(5,), stride=(1,), padding=valid)
       (1): ReLU()
       (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
       (3): Conv1d(10, 10, kernel_size=(3,), stride=(1,), padding=valid)
       (4): ReLU()
       (5): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
     )
   )
   (recurrent_block): RecurrentBlock(
     (layers): LSTM(10, 10, num_layers=10, batch_first=True, bidirectional=True)
   )
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=20, out_features=10, bias=True)
     )
   )
 ),
 torch.Size([10, 10]))

We encourage the user to play with different arguments to these architectures, and not just because we selfishly want them to help us find any bugs that have missed our testing ;)

#### Published architectures

Sometimes, even having to specifiy the arguments to the layers you want is too much. We get it. We've been there. That's why we've built in set of models that are designed for specific tasks. We have also constructed several published architectures that often represent specific configurations of these basic architectures and made them accessible to users through single function calls. One example is the transcription factor binding classifiers. These models are available in the `eugene.models.zoo._tf_binding_predictors` module.

In [81]:
from eugene.models.zoo._tf_binding_predictors import DeepBind, Kopp21CNN

At a minimum, each architecture expects an input sequence length (`input_len`) and an output dimension (`output_dim`). Other required and optional arguments are model specific, but each model follows the same contract as before. That is, each model is a class that inherits from `torch.nn.Module` and includes a `forward` method.

In [85]:
model = DeepBind(
    input_len=100,
    output_dim=1,
    mode="rbp"
)
model_out = model(x)
model, model_out.shape

(DeepBind(
   (conv1d_tower): Conv1DTower(
     (layers): Sequential(
       (0): Conv1d(4, 16, kernel_size=(16,), stride=(1,), padding=valid)
       (1): ReLU()
       (2): Dropout(p=0.25, inplace=False)
     )
   )
   (max_pool): MaxPool1d(kernel_size=85, stride=85, padding=0, dilation=1, ceil_mode=False)
   (avg_pool): AvgPool1d(kernel_size=(85,), stride=(85,), padding=(0,))
   (dense_block): DenseBlock(
     (layers): Sequential(
       (0): Linear(in_features=32, out_features=32, bias=True)
       (1): ReLU()
       (2): Dropout(p=0.25, inplace=False)
       (3): Linear(in_features=32, out_features=1, bias=True)
     )
   )
 ),
 torch.Size([10, 1]))

We leave it as an exercise to the reader to explore the other published architectures in the model zoo.

### LightningModules for training architectures

One thing you may be thinking to yourself if you've made it this far is, "Wait, how do I make sure I train the DeepBind architecture as the creator intended" (e.g. for binary classification). It is true, there is nothing in the way we've written any of the built-in architectures requiring you to train them in any particular way (again, maybe we are lumpers). Instead, we've written that contract into something called a LightningModule from PyTorch Lightning! Don't worry, you don't have to know anything about PyTorch Lightning to use these modules, just on how they are used in EUGENe. As a starting place, we can grab LightningModules directly from the `eugene.models` module.

In [ ]:
from eugene import models

Each LightningModules is meant to do the following:

1. define the types of architectures it can train
2. standardize the way a user interacts with those architectures in EUGENe
3. reduce boilerplate PyTorch and PyTorch Lighting code. 

For example, we implemented SequenceModule to expect an architecture that ingests a single tensor (usually one-hot encoded DNA sequences) as input and outputs a single tensor. The SequenceModule defines how this class of models should be trained (including the loss function and optimizer), what metrics should be reported, and how inference should be handled. As a result, any PyTorch model that follows this contract can be trained using SequenceModule. 

In [88]:
# We first need to create a model architecture that we can pass to the module. We kept it simple with DeepBind here
arch = DeepBind(
    input_len=100,
    output_dim=1
)
arch

DeepBind(
  (conv1d_tower): Conv1DTower(
    (layers): Sequential(
      (0): Conv1d(4, 16, kernel_size=(16,), stride=(1,), padding=valid)
      (1): ReLU()
      (2): Dropout(p=0.25, inplace=False)
    )
  )
  (max_pool): MaxPool1d(kernel_size=85, stride=85, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool1d(kernel_size=(85,), stride=(85,), padding=(0,))
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.25, inplace=False)
      (3): Linear(in_features=32, out_features=1, bias=True)
    )
  )
)

In [90]:
# Next we pass the architecture to the module, along with several arguments that we want to use to train the model
module = models.SequenceModule(
    arch=arch,
    task="binary_classification",
    loss_fxn="bce",
    optimizer="adam",
    metric="auroc"
)
module

SequenceModule(
  (arch): DeepBind(
    (conv1d_tower): Conv1DTower(
      (layers): Sequential(
        (0): Conv1d(4, 16, kernel_size=(16,), stride=(1,), padding=valid)
        (1): ReLU()
        (2): Dropout(p=0.25, inplace=False)
      )
    )
    (max_pool): MaxPool1d(kernel_size=85, stride=85, padding=0, dilation=1, ceil_mode=False)
    (avg_pool): AvgPool1d(kernel_size=(85,), stride=(85,), padding=(0,))
    (dense_block): DenseBlock(
      (layers): Sequential(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.25, inplace=False)
        (3): Linear(in_features=32, out_features=1, bias=True)
      )
    )
  )
  (train_metric): BinaryAUROC()
  (val_metric): BinaryAUROC()
  (test_metric): BinaryAUROC()
)

And voila! We have a architecture wrapped in a SequenceModule that is now ready for training! Let's break down the arguments in the code above.
1. `task` - the task we are training the model for. This is used to determine the loss function and metrics to use when we don't pass them in explicitly.
2. `loss_fxn` - the loss function to use for training. If not provided, the loss function is determined based on the task.
3. `optimizer` - the optimizer to use for training.
4. `metric` - the metric to use for training. If not provided, the metric is determined based on the task.

Remember, the `SequenceModule` is expecting an architecture that follows the contract we described above. DeepBind, as well as most of the other built-in architectures, follow this contract. If we pass in an architecture that doesn't follow this contract, however, we will likely get an error when we try to train the model.

We have also implemented a ProfileModule that handles BPNet style training, where the model has multiple output tensors (“heads”), can take in optional control inputs, and uses multiple loss functions. We only have one architecture that follows this contract as of right now (BPNet), but who knows maybe other architectures will follow this contract in the future.                                                                                                                         

In [91]:
from eugene.models.zoo import BPNet

ImportError: cannot import name 'BPNet' from 'eugene.models.zoo' (/cellar/users/aklie/projects/ML4GLand/EUGENe/eugene/models/zoo/__init__.py)

In [ ]:
arch = BPNet(
    input_len=2114,
    output_dim=1000,
    n_outputs=2,
    n_control_tracks=2, 
    trimming=(2114 - 1000) // 2,
    name="BPNet"
)

In [92]:
from eugene.models import ProfileModule

ImportError: cannot import name 'ProfileModule' from 'eugene.models' (/cellar/users/aklie/projects/ML4GLand/EUGENe/eugene/models/__init__.py)

In [ ]:
module = ProfileModule(arch=arch)
module

## Using config files

By now, hopefully you have a pretty good feel how architectures work in EUGENe and we can move into some nuances.

It can be quite cumbersome to drag around the arguments you need to build a model to every place you need to instantiate it. It can also be quite cumbersome to have to remember all the arguments you need to build a model. That's why we've built in a config file system that allows you to save and load model configurations. Let's start with an arleady generated config for a CNN model.

In [115]:
import importlib
import os
import torch
import yaml
from eugene import settings

In [116]:
# We can make use of a EUGENe helper for loading from a config
load_config("/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/notebooks/tutorials/simple_cnn.yaml")

SequenceModule(
  (arch): CNN(
    (conv1d_tower): Conv1DTower(
      (layers): Sequential(
        (0): Conv1d(4, 32, kernel_size=(13,), stride=(1,), padding=valid)
        (1): ReLU()
        (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Dropout(p=0.3, inplace=False)
        (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (dense_block): DenseBlock(
      (layers): Sequential(
        (0): Linear(in_features=1408, out_features=64, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Linear(in_features=64, out_features=1, bias=True)
      )
    )
  )
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
)

Let's take a look at the config directly

In [117]:
!cat /cellar/users/aklie/projects/ML4GLand/EUGENe_paper/notebooks/tutorials/simple_cnn.yaml

module: SequenceModule
model:
  model_name: simple_cnn
  arch_name: CNN
  arch:
    input_len: 100
    output_dim: 1
    conv_kwargs:
      input_channels: 4
      conv_channels: [32]
      conv_kernels: [13]
      conv_strides: [1]
      pool_kernels: [2]
      pool_strides: [2]
      dropout_rates: 0.3
      batchnorm: True
      activations: relu
    dense_kwargs:
      hidden_dims: [64]
      dropout_rates: 0.2
      batchnorm: True
task: regression
loss_fxn: mse
optimizer: adam
optimizer_lr: 0.001


We can see that it looks a lot like the arguments we passed into the CNN architecture. It's really the same process either way, the config file just has a little more persistence.

Configs work similarly for the published architectures. Let's load in a config for the DeepBind architecture.

In [121]:
models.load_config("/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/configs/ray13/deepbind.yaml")

SequenceModule(
  (arch): DeepBind(
    (conv1d_tower): Conv1DTower(
      (layers): Sequential(
        (0): Conv1d(4, 16, kernel_size=(16,), stride=(1,), padding=valid)
        (1): ReLU()
        (2): Dropout(p=0.25, inplace=False)
      )
    )
    (max_pool): MaxPool1d(kernel_size=85, stride=85, padding=0, dilation=1, ceil_mode=False)
    (avg_pool): AvgPool1d(kernel_size=(85,), stride=(85,), padding=(0,))
    (dense_block): DenseBlock(
      (layers): Sequential(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.25, inplace=False)
        (3): Linear(in_features=32, out_features=1, bias=True)
      )
    )
  )
  (train_metric): R2Score()
  (val_metric): R2Score()
  (test_metric): R2Score()
)

In [122]:
!cat /cellar/users/aklie/projects/ML4GLand/EUGENe_paper/configs/ray13/deepbind.yaml

module: SequenceModule
model:
  model_name: deepbind_st
  arch_name: DeepBind
  arch:
    input_len: 41
    output_dim: 1
task: regression
loss_fxn: mse
optimizer: adam
optimizer_lr: 0.005
scheduler_kwargs:
  patience: 2


Note, that when you build a SequenceModule with a config, you must use a built-in architecture.

Finally, you don't have to add the SequenceModule specific arguments to the config if you don't want to. You can specify just the architecture

In [123]:
models.load_config("/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/configs/ray13/deepbind_arch.yaml")

DeepBind(
  (conv1d_tower): Conv1DTower(
    (layers): Sequential(
      (0): Conv1d(4, 16, kernel_size=(16,), stride=(1,), padding=valid)
      (1): ReLU()
      (2): Dropout(p=0.25, inplace=False)
    )
  )
  (max_pool): MaxPool1d(kernel_size=85, stride=85, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool1d(kernel_size=(85,), stride=(85,), padding=(0,))
  (dense_block): DenseBlock(
    (layers): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.25, inplace=False)
      (3): Linear(in_features=32, out_features=1, bias=True)
    )
  )
)

## Using custom architectures

If I've said it twice, all say it three times, architectures in EUGENe are just classes that inherit from `torch.nn.Module` and include a `forward` method. This means that you can grab PyTorch models from anywhere and use them in EUGENe. Let's build an new architecture from scratch and plug it into a SequenceModule.

In [134]:
# You could imagine building this a ton of different ways, but here is a simple example
import torch.nn.functional as F
class SmallCNN(nn.Module):
    def __init__(self):
        super(SmallCNN, self).__init__()

        # Set the attributes
        self.input_len = 100
        self.output_dim = 1

        # Create the blocks
        self.conv1 = nn.Conv1d(4, 30, 21)
        self.relu  = nn.ReLU()
        
        self.dense = nn.Linear(30, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = F.max_pool1d(x, x.size()[-1]).flatten(1, -1)
        x = self.dense(x)
        return x

In [135]:
# Now we can pass this to the module
models.SequenceModule(
    arch=SmallCNN(),
    task="binary_classification",
    loss_fxn="bce",
    optimizer="adam",
    metric="auroc"
)

SequenceModule(
  (arch): SmallCNN(
    (conv1): Conv1d(4, 30, kernel_size=(21,), stride=(1,))
    (relu): ReLU()
    (dense): Linear(in_features=30, out_features=1, bias=True)
  )
  (train_metric): BinaryAUROC()
  (val_metric): BinaryAUROC()
  (test_metric): BinaryAUROC()
)

And away we go!

## Using custom LigthningModules

We can also do the reverse process by plugging in a built-in architecture to a custom LightningModule. Let's use the EvoAug RobustModel LightningModule, and plug in the DeepBind architecture.

In [124]:
import torch
from evoaug import evoaug
from evoaug_analysis import utils

In [137]:
# Load the deepbind model that was trained in the evo_aug paper
deepbind = DeepBind(249, 2)
loss = torch.nn.MSELoss()
optimizer_dict = utils.configure_optimizer(deepbind, lr=0.001, weight_decay=1e-6, decay_factor=0.1, patience=5, monitor='val_loss')
model = evoaug.RobustModel(
    deepbind, 
    criterion=loss, 
    optimizer=optimizer_dict, 
    augment_list=[]
)
model

RobustModel(
  (model): DeepBind(
    (conv1d_tower): Conv1DTower(
      (layers): Sequential(
        (0): Conv1d(4, 16, kernel_size=(16,), stride=(1,), padding=valid)
        (1): ReLU()
        (2): Dropout(p=0.25, inplace=False)
      )
    )
    (max_pool): MaxPool1d(kernel_size=85, stride=85, padding=0, dilation=1, ceil_mode=False)
    (avg_pool): AvgPool1d(kernel_size=(85,), stride=(85,), padding=(0,))
    (dense_block): DenseBlock(
      (layers): Sequential(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.25, inplace=False)
        (3): Linear(in_features=32, out_features=2, bias=True)
      )
    )
  )
  (criterion): MSELoss()
)

Once again, away we go!